# 4.4 MLP - Stochastic Gradient Descent Method

In [18]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 3. 19. 2020 - 
 Title: Artificial Intelligence_Project 2
 Professor: Seung-Hoon Na'''
  
import numpy as np
import random
import matplotlib.pyplot as plt

# mnist data

In [3]:
from sklearn import datasets
mnist = datasets.fetch_openml('mnist_784')

In [4]:
(mnist.data.shape, mnist.target.shape)

((70000, 784), (70000,))

In [5]:
np.unique(mnist.target)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

In [6]:
data=list(zip(mnist.data,mnist.target))
#data=np.concatenate((mnist.data,mnist.target),axis=1)
data=np.array(data)
x=data[:,0]
x=np.array(x)
#y=data[:,1]
print(x.shape)
#print(y)

(70000,)


In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
mnist.target = encoder.fit_transform(mnist.target.reshape(-1, 1))

In [8]:
         
train_idx=int(data.shape[0]*0.85)
dev_idx=int(data.shape[0]*0.05)
test_idx=int(data.shape[0]*0.1)

data=np.array(mnist.data)
target=np.array(mnist.target)

train_x=data[0:train_idx,:]
test_x=data[train_idx:train_idx+test_idx,:]
dev_x=data[train_idx+test_idx:train_idx+test_idx+dev_idx,:]


train_y=target[0:train_idx,:]
test_y=target[train_idx:train_idx+test_idx,:]
dev_y=target[train_idx+test_idx:train_idx+test_idx+dev_idx,:]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(dev_x.shape)
print(dev_y.shape)

(59500, 784)
(59500, 10)
(7000, 784)
(7000, 10)
(3500, 784)
(3500, 10)


In [9]:
combined = list(zip(train_x, train_y))
np.random.shuffle(combined)
train_x[:], train_y[:] = zip(*combined)

# Make relu class

In [10]:
class relu():
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, affine, dout):
        mask = (affine<=0)
        dout[mask]= 0
        return dout

### MLP

In [11]:
class MLP():
    def __init__(self):
        self.a = 0
        
    def forward(self,W, X, b):
        
        self.a = np.dot(X,W) + b
        
        return self.a

# Make softmax class

In [12]:
class softmax():    
    def __init__(self):
        self.exp_o = 0
        self.sum_exp_o = 0
        self.delta = 0
        
    def forward(self,h):
        self.delta = np.max(h)
        
        self.exp_o = np.exp(h - self.delta)
        self.sum_exp_o = np.sum(self.exp_o)
        self.softmax = self.exp_o / self.sum_exp_o
        
        return self.softmax

# Make Cross_entropy class

In [13]:
class CE():
    def __init__(self):
        self.result = 0
        
    def forward(self, softMAX, y):
        # cross_entropy + 마이너스 무한대가 생기지 않도록 만듦
        delta = 1e-7
        
        return - np.sum(y * np.log(softMAX + delta))
        
    def backward(self, y, softMAX):
        batch_size = y.shape[0]
        dJdR = (softMAX-y) / batch_size
        return dJdR

# 4.4

In [17]:
model = MLP()
ce = CE()
Relu = relu()
SOFTMAX = softmax()

layer=[]
param_w = []
param_b = []
affine = []

grad_w = []
grad_b = []
grad_affine = []

L = int(input('Layer: '))

for i in range(L):
    print(i+1,'th Dimenson', end = ' ') 
    d=int(input(':'))
    
    layer.append(d)
    
length = len(layer)

for i in range(length):
    param_w.append(0)
    param_b.append(0)
    grad_w.append(0)
    grad_b.append(0)
for i in range(length+1):
    affine.append(0)
    grad_affine.append(0)

minibatch_size = int(input('Minibatch size(10~500): '))
epoch_size = int(input('epoch size: '))
#iter_per_epoch= max(X[:].shape[0] / minibatch_size, 1)

acc_list = []
test_acc_list = []

def accuracy(x, t, J):
    acc_y = np.argmax(J, axis=1)
    if t.ndim !=1 : t = np.argmax(t,axis=1)
        
    accuracy = np.sum(acc_y == t) / float(x.shape[0])
    
    return accuracy

def learning(X,y, minibatch_size, epoch_size, learning_rate):
    d = np. random.randn(10)
    U = np. random.randn(layer[length-1],y.shape[1])
    for e in range(1,epoch_size+1):
        print('*************',e,'번차 epoch *************')
        
        combined = list(zip(X, y))
        np.random.shuffle(combined)
        X[:], y[:] = zip(*combined)

        number_minibatch= np.int(np.ceil(X.shape[0] / minibatch_size))
        
        for n in range(1, number_minibatch+1):
            X_temp=X[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]
            X_temp = (X_temp / 255 * 0.99) + 0.01
            y_temp=y[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]
            
            ########################## MLP ########################## 
            for i in range(length):
                #print('-------------',i+1,'번째 layer-------------')
                if (i == 0):
                    x = X_temp
                    W = np. random.randn(784,layer[i])
                    b = np. random.randn(layer[i])
                    a = model.forward(W, x, b)
                    x = Relu.forward(a)
                    
                    param_w[i] = W
                    param_b[i] = b
                    affine[i] = a
                else:
                    W = np. random.randn(np.array(x).shape[1],layer[i])
                    b = np. random.randn(layer[i])
                    a = model.forward(W, x, b)
                    Relu.forward(a)

                    param_w[i] = W
                    param_b[i] = b
                    affine[i] = a
                    
                    x = Relu.forward(a)
                    
            R = np.dot(x, U) + d
            affine[length] = R
            h  = Relu.forward(R)
            o = SOFTMAX.forward(h) # softmax
            J = ce.forward(o, y_temp) # cross_Entropy
            
            ############# BACKWARD #############
            dJdh = ce.backward(y_temp,o) #softmax & cross_Entropy loss
            dJdR = Relu.backward(affine[length],dJdh)
            dJdU = np.dot(np.transpose(x),dJdR)
            dJd_ = np.dot(dJdR,np.transpose(U))
            dJdd = np.ones_like(d)

            param_w.reverse()
            param_b.reverse()
            affine.reverse()
            affine[length] = X_temp

            for x in range(length):
                #print('------------- from back',x+1,'번째 backpropagation-------------')
                if (x != 0):
                    dJd_ = Relu.backward(affine[x],dJd_)
                    dJd_ = np.dot(dJd_,np.transpose(param_w[x-1]))
    
                dJdw = np.dot(np.transpose(affine[x+1]),dJd_)
                dJdb = np.ones_like(param_b[x])
                grad_w.append(dJdw)
                grad_b.append(dJdb)
                
            #print('------------- from back',length,'번째 backpropagation-------------')
            dJd_ = np.dot(dJd_,np.transpose(param_w[length-1]))
            dJdw = np.dot(np.transpose(X_temp),dJd_)
            dJdb = np.ones_like(param_b[length-1])
            grad_w.append(dJdw)
            grad_b.append(dJdb)
            
            for key in range(length-1):
                param_w[key] = param_w[key]- learning_rate * grad_w[key]
                param_b[key] = param_b[key]- learning_rate * grad_b[key]
            U = U - learning_rate * dJdU
            d = d - learning_rate * dJdd
            
        param_w.reverse()
        param_b.reverse()

        ### J출력 ###
        for r in range(length):
            if (r == 0):
                #### 첫번째 layer ####
                x = X[:]
                a = model.forward(param_w[r], x, param_b[r])
                x = Relu.forward(a)
                #print(x)
            else:
                a = model.forward(param_w[r], x, param_b[r])
                x = Relu.forward(a)
                #print(x)        
        
        R = np.dot(x, U) + d
        h  = Relu.forward(R)
        o = SOFTMAX.forward(h) # softmax
        J = ce.forward(o, y) # cross_Entropy
        
        acc = accuracy(X, y, o)
        acc_list.append(acc)
    
        print('최종')
        print(J)
        print('정확도')
        print(acc)
train_ = learning(train_x, train_y, minibatch_size, epoch_size, 10)

Layer:  3


1 th Dimenson 

: 4


2 th Dimenson 

: 3


3 th Dimenson 

: 4
Minibatch size(10~500):  100
epoch size:  5


************* 1 번차 epoch *************
최종
958940.7332113304
정확도
0.11376470588235293
************* 2 번차 epoch *************
최종
959026.691232021
정확도
0.11332773109243698
************* 3 번차 epoch *************
최종
959026.6912320209
정확도
0.1166218487394958
************* 4 번차 epoch *************
최종
959026.6912320205
정확도
0.11700840336134453
************* 5 번차 epoch *************
최종
959026.6912320212
정확도
0.11704201680672269
